# FIM Generation using Arcgis Pro

**Author(s):** 

<ul style="line-height:1.5;">
<li>Nana Oye Djan <a href="mailto:ndjan@andrew.cmu.edu">(ndjan@andrew.cmu.edu)</a></li>
<li>Kayode Adebayo <a href="mailto:kayode.adebayo@jacks.sdstate.edu">(kayode.adebayo@jacks.sdstate.edu)</a></li>
<li>Saide Zand <a href="mailto:szand@crimson.ua.edu">(szand@crimson.ua.edu)</a></li>
</ul>

**Last Updated:** 
17th July 2025

**Purpose:**

This notebook provides code to translate shapefiles either from our get_nwm_sr_forecast notebook or get_nwm_ret_incidents into HAND-FIMs.

**Description:**

This notebook produces HAND-FIMs using shapefiles of discharge values for each reach in Travis County and Synthetic Rating Curves (SRC) using the method outlined in Dean Djokic's resource found <a href="https://www.hydroshare.org/resource/23aa7866ab614687811bb70ffb13fcfe/">(here)</a></li>.

**Data Description:**

This notebook uses the rating curves found in the geodatabase in <a href="https://www.hydroshare.org/resource/23aa7866ab614687811bb70ffb13fcfe/">(this ArcGIS project)</a></li>. It also uses shapefiles generated by our get_nwm_sr_forecast notebook or get_nwm_ret_incidents notebooks.

**Software Requirements:**

This notebook is run in ArcGIS Pro and requires the following packages
> arcpy \
    pandas \
    os \
    uuid \
    traceback \
    datetime \
    collections \
    dateutil.parser \
It also requires the ArcHydro toolbox for the Q to H interpolation using the rating curves.

**Disclosure**
The code contained in this notebook was partially created and revised by ChatGPT, an AI language model developed by OpenAI

In [ ]:
import arcpy
import pandas as pd
import os
import uuid
import traceback
import datetime as dt
from collections import defaultdict
import datetime as dt
from dateutil.parser import parse as parse_dt

# === Set your environment ===
arcpy.env.workspace = r"C:\Users\kbadebayo\Documents\ArcGIS\Projects\Theme4DataRevised_testing\Theme4DataRevised_testing"
arcpy.env.overwriteOutput = True

# Auxiliary data needed
#rating_curve_table = r"C:\Mac\Home\Documents\ArcGIS\Projects\Theme4DataRevised\Theme4Data.gdb\RatingCurves"
rating_curve_table = r"C:\Users\kbadebayo\Documents\ArcGIS\Projects\Theme4DataRevised_testing\Theme4DataRevised_testing\Theme4Data.gdb\RatingCurves"
rating_curve= rating_curve_table 
#flood_stack = r"C:\Mac\Home\Documents\ArcGIS\Projects\Theme4DataRevised\Theme4Data.gdb\TravisFIM"
flood_stack = r"C:\Users\kbadebayo\Documents\ArcGIS\Projects\Theme4DataRevised_testing\Theme4DataRevised_testing\Theme4Data.gdb\TravisFIM"
output_gdb = r"C:\Users\kbadebayo\Documents\ArcGIS\Projects\Theme4DataRevised_testing\Theme4DataRevised_testing\Theme4Data.gdb"

# Shapefile with flows
shp_file = r"C:\Users\kbadebayo\Documents\ArcGIS\Projects\shape_nana\sr_nwm_tc_max10hr.shp"

### For our 3 FIMs

folder = r"C:\Users\kbadebayo\Documents\ArcGIS\Projects\discharge_shapefiles_2"

def safe_add_field(table, field_name, field_type):
    """Add field only if it doesn't already exist."""
    if field_name.lower() not in [f.name.lower() for f in arcpy.ListFields(table)]:
        arcpy.AddField_management(table, field_name, field_type)

def process_fim_flow(in_fc, flow_field, out_name):
    # clear any leftover locks
    arcpy.ClearWorkspaceCache_management()

    # 1) Bring your source in as a layer (no locks on the file itself)
    fc_lyr = arcpy.MakeFeatureLayer_management(in_fc, "fc_tmp").getOutput(0)

    # 2) add & calculate Q_cfs on that layer
    safe_add_field(fc_lyr, "Q_cfs", "DOUBLE")
    arcpy.CalculateField_management(
        fc_lyr, "Q_cfs", f"!{flow_field}! * 1", "PYTHON3"
    )

    # 3) copy the layer into scratch/in_memory to work on it
    scratch = arcpy.env.scratchGDB
    working = arcpy.CopyFeatures_management(
        fc_lyr, os.path.join(scratch, f"{out_name}_wrk")
    ).getOutput(0)

    # 4) delete the temp layer immediately
    arcpy.Delete_management(fc_lyr)
    
    # 2) interpolate Q → H
    arcpy.ImportToolbox(
        r"C:\Program Files\ArcGIS\Pro\Resources\ArcToolbox\Toolboxes\Arc_Hydro_Tools_Pro.tbx",
        "hydro"
    )
    arcpy.interpolatefromlookuptablewithfieldmappings_hydro(
        working, rating_curve,
        "Q to H", "Q_cfs:discharge_","H:stage_m",
        "HydroID","HydroID"
    )
    # 3) compute HIndex only where H is non-null
    lyr = arcpy.MakeFeatureLayer_management(working, "hasH", "H IS NOT NULL").getOutput(0)
    arcpy.CalculateField_management(
        lyr, "HIndex",
        "int(round(!H! *3.28) + 1)", "PYTHON3"
    )
    arcpy.Delete_management(lyr)

    # 4) spatial join to flood polygons
    out_fc = os.path.join(output_gdb, out_name)
    if arcpy.Exists(out_fc):
        arcpy.Delete_management(out_fc)

    flood_lyr  = arcpy.MakeFeatureLayer_management(flood_stack, "flood_tmp").getOutput(0)
    stream_lyr = arcpy.MakeFeatureLayer_management(working,     "strm_tmp").getOutput(0)
    
    # cast join keys to LONG
    for fld in ("HydroID", "HIndex"):
        field_type = "LONG" if fld == "HydroID" else "DOUBLE"
        safe_add_field(stream_lyr, fld + "_num", field_type)
        arcpy.CalculateField_management(
            stream_lyr, fld + "_num", f"!{fld}!", "PYTHON3"
    )


    arcpy.analysis.SpatialJoin(
        flood_lyr, stream_lyr, out_fc,
        "JOIN_ONE_TO_MANY", "KEEP_COMMON",
        match_fields=[["HydroID_num","HydroID"], ["HIndex_num","HIndex"]]
    )
    
    # cleanup
    for tmp in (flood_lyr, stream_lyr, working):
        arcpy.Delete_management(tmp)

    arcpy.Delete_management(working)

    print(f"✅  {out_name} written to {output_gdb}")



# ─────────────────────────────────────────────────────────────────────────────
#  Main driver: loop the folder
# ─────────────────────────────────────────────────────────────────────────────
arcpy.env.overwriteOutput = True
arcpy.env.workspace     = folder



lwc =r"C:\Users\kbadebayo\Documents\ArcGIS\Projects\Datasets\lowwatercrossing_clean\clean_LWC.shp"

def classify_lwc(fim_fc, output_name):
    # Path to original LWC
    lwc_input = r"C:\Users\kbadebayo\Documents\ArcGIS\Projects\Datasets\lowwatercrossing_clean\clean_LWC.shp"
    
    # Output path for classified LWC
    lwc_out = os.path.join(output_gdb, f"{output_name}_LWC")

    # Copy the LWC shapefile into the output geodatabase
    arcpy.FeatureClassToFeatureClass_conversion(lwc_input, output_gdb, f"{output_name}_LWC")

    # Remove "Condition" if it already exists
    fields = [f.name for f in arcpy.ListFields(lwc_out)]
    if "Condition" in fields:
        arcpy.DeleteField_management(lwc_out, "Condition")

    # Add new Condition field
    arcpy.AddField_management(lwc_out, "Condition", "TEXT", field_length=10)

    # Make layer from LWC and FIM
    fim_lyr = arcpy.MakeFeatureLayer_management(fim_fc, "fim_lyr").getOutput(0)
    lwc_lyr = arcpy.MakeFeatureLayer_management(lwc_out, "lwc_lyr").getOutput(0)

    # Select only LWC points that intersect the FIM
    arcpy.SelectLayerByLocation_management(
        lwc_lyr, "INTERSECT", fim_lyr, selection_type="NEW_SELECTION"
    )

    # Set selected LWC points to "Flooded"
    arcpy.CalculateField_management(lwc_lyr, "Condition", '"Flooded"', "PYTHON3")

    # Switch selection and set others to "Safe"
    arcpy.SelectLayerByAttribute_management(lwc_lyr, "SWITCH_SELECTION")
    arcpy.CalculateField_management(lwc_lyr, "Condition", '"Safe"', "PYTHON3")

    # Cleanup
    arcpy.Delete_management(fim_lyr)
    arcpy.Delete_management(lwc_lyr)

    print(f"✅  {output_name}_LWC written to {output_gdb}")


def classify_address_points(fim_fc, output_name):
    # Path to original Address Points shapefile
    address_input = r"C:\Users\kbadebayo\Documents\ArcGIS\Projects\Datasets\addresspoint\addresspoint.shp"
    
    # Output path for classified Address Points
    address_out = os.path.join(output_gdb, f"{output_name}_ADDR")

    # Copy to output geodatabase
    arcpy.FeatureClassToFeatureClass_conversion(address_input, output_gdb, f"{output_name}_ADDR")

    # Remove "Condition" if it exists
    fields = [f.name for f in arcpy.ListFields(address_out)]
    if "Condition" in fields:
        arcpy.DeleteField_management(address_out, "Condition")

    # Add new "Condition" field
    arcpy.AddField_management(address_out, "Condition", "TEXT", field_length=10)

    # Make layers
    fim_lyr = arcpy.MakeFeatureLayer_management(fim_fc, "fim_lyr_addr").getOutput(0)
    addr_lyr = arcpy.MakeFeatureLayer_management(address_out, "addr_lyr").getOutput(0)

    # Select points that intersect FIM
    arcpy.SelectLayerByLocation_management(
        addr_lyr, "INTERSECT", fim_lyr, selection_type="NEW_SELECTION"
    )
    arcpy.CalculateField_management(addr_lyr, "Condition", '"Flooded"', "PYTHON3")

    # Select the safe points
    arcpy.SelectLayerByAttribute_management(addr_lyr, "SWITCH_SELECTION")
    arcpy.CalculateField_management(addr_lyr, "Condition", '"Safe"', "PYTHON3")

    # Cleanup
    arcpy.Delete_management(fim_lyr)
    arcpy.Delete_management(addr_lyr)

    print(f"✅  {output_name}_ADDR written to {output_gdb}")




for shp in arcpy.ListFiles("*.shp"):
    full = os.path.join(folder, shp)

    # 1-hour forecasts → two passes: ml_dis then wc_dis
    if "sr_nwm_tc_1hour" in shp.lower():
        out_name = "FIM_1hour_ml"
        process_fim_flow(full, "ml_dis", out_name)
        classify_lwc(os.path.join(output_gdb, out_name), out_name)
        #classify_address_points(os.path.join(output_gdb, out_name), out_name)
        out_name = "FIM_1hour_wc"
        process_fim_flow(full, "wc_dis", out_name)
        classify_lwc(os.path.join(output_gdb, out_name), out_name)
        #classify_address_points(os.path.join(output_gdb, out_name), out_name)

    # 2-hour forecasts → two passes: ml_dis then wc_dis
    elif "sr_nwm_tc_2hour" in shp.lower():
        out_name = "FIM_2hour_ml"
        process_fim_flow(full, "ml_dis", out_name)
        classify_lwc(os.path.join(output_gdb, out_name), out_name)
        #classify_address_points(os.path.join(output_gdb, out_name), out_name)
        out_name = "FIM_2hour_wc"
        process_fim_flow(full, "wc_dis", out_name)
        classify_lwc(os.path.join(output_gdb, out_name), out_name)
        #classify_address_points(os.path.join(output_gdb, out_name), out_name)

    # max-10-hour → single pass on streamflow
    elif "sr_nwm_tc_max10hr" in shp.lower():
        out_name = "FIM_10hour_wc"
        #process_fim_flow(full, "streamflow", "FIM_max_10hour")
        process_fim_flow(full, "streamflow", out_name)
        classify_lwc(os.path.join(output_gdb, out_name), out_name)
        #classify_address_points(os.path.join(output_gdb, out_name), out_name)